# Importing libraries

In [160]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import balanced_accuracy_score

from fastquant import backtest, get_stock_data
from fastquant.strategies import TernaryStrategy
from matplotlib import pyplot as plt
import matplotlib.dates

# Importing data and last cleanup

In [161]:
df = pd.read_csv("cleaned_data.csv")

In [162]:
df = df.loc[df["Decision14"] != -1]
df2 = df[['Support', 'Resistance','Hammer', 'ZeroCross', 'SignalCross', 'Decision14']]
#df2 = df[['Resistance', 'Williams%R', 'SignalCross' ]]

# Splitting the data

In [163]:
train = df2[0:692] #from 2016 to 2018
test = df2[692:] #from 2019 to 2020

In [164]:
x_train = train.iloc[:,:-1].to_numpy()
x_test = test.iloc[:,:-1].to_numpy()
y_train = train.iloc[:,-1].to_numpy()
y_test = test.iloc[:,-1].to_numpy()


In [165]:
test

,Support,Resistance,Hammer,ZeroCross,SignalCross,Decision14
725,0,0,0.0,0.0,1.0,1
726,0,0,0.0,0.0,1.0,1
727,0,0,0.0,0.0,1.0,1
728,0,0,0.0,1.0,1.0,1
729,0,0,0.0,1.0,1.0,1
...,...,...,...,...,...,...
1023,0,0,0.0,0.0,0.0,1
1024,0,0,0.0,0.0,0.0,1
1025,0,0,0.0,0.0,0.0,1
1026,0,0,0.0,0.0,0.0,1


# Getting the predictions

In [166]:
clf = ComplementNB()
clf.fit(x_train,y_train)
pred = clf.predict(x_test)
print(balanced_accuracy_score(y_test,pred))
print(clf.score(x_test,y_test))

0.5579926302860151
0.528052805280528


# Adding hold to the predictions
## repeated signals will be transformed to hold
## to be consistent with the fastquant libraries, sell = -1, hold = 0, buy = 1

In [167]:
pred = np.where(pred == 0,-1, pred) #changing sell signals to -1
pred

array([ 1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1

In [168]:
def add_hold(arr):
    last_decision = arr[0]
    decision = [last_decision]
    for i in range(1,len(arr)):
        if arr[i] == last_decision:
            decision.append(0)
        else:
            last_decision = arr[i]
            decision.append(last_decision)
    return decision

In [169]:
pred2 = add_hold(pred)

In [170]:
fastquant_data = df[692:]

In [171]:
fastquant_data["custom"] = pred2

# Pre processing fastquant_data according to the library

In [172]:
#setting the data as indexes 
fastquant_data["dt"] = fastquant_data["Date"]
fastquant_data.drop("Date", axis = 1)
fastquant_data.set_index("dt", inplace=True)

In [173]:
#keeping the relevant data
fastquant_data = fastquant_data[['Close','Open','Low','High', 'custom']]

# Backtesting the strategy

In [174]:
fastquant_data

,Close,Open,Low,High,custom
dt,,,,,
2019-01-02,23.42,21.95,21.68,23.55,1
2019-01-03,23.99,23.32,23.16,24.16,0
2019-01-04,24.06,24.18,23.82,24.27,0
2019-01-07,24.44,24.18,24.04,25.23,-1
2019-01-08,24.29,24.72,24.11,24.74,0
...,...,...,...,...,...
2020-03-16,13.09,12.91,12.55,14.10,0
2020-03-17,13.00,13.60,13.00,13.84,0
2020-03-18,11.29,11.79,10.87,12.29,0


In [175]:
backtest(TernaryStrategy,fastquant_data, init_cash = 2342)

Starting Portfolio Value: 2342.00
2020-03-20, ===Global level arguments===
2020-03-20, init_cash : 2342
2020-03-20, buy_prop : 1
2020-03-20, sell_prop : 1
2020-03-20, commission : 0.0075
2020-03-20, stop_loss : 0
2020-03-20, stop_trail : 0
===Strategy level arguments===
Buy Int:  1
Sell Int:  -1
2020-03-20, Final Portfolio Value: 3240.2092000000007
2020-03-20, Final PnL: 898.21
Time used (seconds): 0.09900569915771484
Number of strat runs: 1
Number of strats per run: 1
Strat names: ['TernaryStrategy']
**************************************************
--------------------------------------------------
Strategy Parameters	init_cash:2342	buy_prop:1	sell_prop:1	commission:0.0075	stop_loss:0	stop_trail:0	execution_type:close	channel:	symbol:	allow_short:False	short_max:1.5	add_cash_amount:0	add_cash_freq:M	buy_int:1	sell_int:-1	custom_column:custom
Returns	rtot:0.32463263044553003	ravg:0.0010713948199522444	rnorm:0.3099533090456241	rnorm100:30.99533090456241
Sharpe	sharperatio:1.1406358098

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (c:\users\frede\appdata\local\programs\python\python39\lib\site-packages\matplotlib\dates.py)